## Exploratory Analysis



In [5]:
%load_ext autoreload
%autoreload 2
import os
from datasets import concatenate_datasets
from pysentimiento.irony import load_datasets, id2label

train, dev, test = load_datasets(lang="es", preprocess=False)


df = concatenate_datasets([train, dev, test]).to_pandas()

df["label"] = df["label"].apply(lambda x: id2label[x])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
df["label"].value_counts(normalize=True) * 100

not ironic    66.677778
ironic        33.322222
Name: label, dtype: float64

In [10]:
import pandas as pd
pd.options.display.max_colwidth = 150

df.sample(10)

,text,topic,lang,label
7011,"@RM_Nalapa @mrwhite939 @AgainJairo Claro y la tierra es plana porque lo dice Vox. Menuda panda de fanáticos, hooligans futboleros...",TIERRAPLANISTAS,es,not ironic
8882,Decenas de denuncias coordinadas usan un informe de la Fundación Franco para frenar la exhumación https://t.co/gVQjSC8B7a vía @eldiarioes Los recu...,FRANCO,es,not ironic
1721,No me sirvió para nada todo fue en vano. Espero mejore la calidad.,"ETECSA, INTERNET, DATOS MÓVILES",cu,not ironic
4268,Pensé que era nota del deforma...,estanciasInfantiles,mx,ironic
7858,Terrible manifestación de ineptitud e ignorancia; trivializar su omisión inexcusable ante uno de los logros más trascendentes en la historia del a...,peliculaRoma,mx,not ironic
4677,"Las abuelas ya criaron hijos, a la vez a muchas ya les tocó trabajar fuera de casa; no es tiempo de dejarlas descansar? O que, vamos a tenerlas tr...",estanciasInfantiles,mx,not ironic
904,"Joan Tardá, ha demostrado una vez mas nuestra capacidad ( los ignorantes nos llaman supremacistas ) de democratas a nivel mundial. Ha demostrado y...",TARDÀ,es,not ironic
8325,"Entre estos tipos, los que dicen que la tierra es plana y que hay que ponerle género X"" a los niños al nacer",tierraPlana,mx,not ironic
1940,"Consideró que fue todo un ""exito"", no sólo se afectó el motivo de la prueba, también fue difícil los correos y la comunicación de voz. El teléfon...","ETECSA, INTERNET, DATOS MÓVILES",cu,ironic
7673,"Es el mejor del mundo RC7, Messi en traje rojo diferente a todos, se volvió excéntrico?????","ESCÁNDALOS DEPORTIVOS, SERIE NACIONAL, FÚTBOL, BÉISBOL",cu,not ironic
